In [1]:
# importamos librerias
import requests
from bs4 import BeautifulSoup

In [2]:
# Obtenemos la pagina web con el metodo requests.get
link_principal = 'https://www.pagina12.com.ar/'
pagina_principal = requests.get(link_principal)
pagina_principal.status_code

200

In [3]:
# parseamos la pagina web
soup = BeautifulSoup(pagina_principal.text,'lxml')
type(soup)

bs4.BeautifulSoup

In [4]:
secciones = soup.find('ul', attrs={'class':'horizontal-list main-sections hide-on-dropdown'}).find_all('li')
    #Busca los tag a para optener el href que contiene el link de la seccion
    #recorremos el array de tag a y obtemos los elementos href que contienen links
href_link_secciones=[link.a.get('href') for link in secciones]
href_link_secciones

['https://www.pagina12.com.ar/secciones/el-pais',
 'https://www.pagina12.com.ar/secciones/economia',
 'https://www.pagina12.com.ar/secciones/sociedad',
 'https://www.pagina12.com.ar/suplementos/cultura-y-espectaculos',
 'https://www.pagina12.com.ar/secciones/deportes',
 'https://www.pagina12.com.ar/secciones/el-mundo',
 'https://www.pagina12.com.ar/secciones/ciencia']

In [5]:
def obtener_notas(soup):
    #Función  que recibe un objeto de BeautifullSoup de una página de una sección y devuelve una lista de URLS a las notas de esa sección
    #Secciones -> creamos un array de todos los elementos con el tag DIV que tengan el atributo class=p12-dropdown-column
    # Selecionamos nuestro primer elemento
    # lista vacia para links de notas de secciones
    URL = []
    
    secciones = soup.find('ul', attrs={'class':'horizontal-list main-sections hide-on-dropdown'}).find_all('li')
    #Busca los tag a para optener el href que contiene el link de la seccion
    #recorremos el array de tag a y obtemos los elementos href que contienen links
    href_link_secciones=[link.a.get('href') for link in secciones]
    #seleccionamos el primer link que corresponde a la seccion el pais
    pagina_seccion_el_pais= requests.get(href_link_secciones[0])
     #parseamos la pagina seccion el pais
    soup_pagina_seccion_el_pais= BeautifulSoup(pagina_seccion_el_pais.text,'lxml')
     #la pagina de articulos de noticias se dividen en 3 sectores
    #Top (1 noticia principal) - Semi(2 noticias semiprincipales) -  articulos (el resto de noticias)

    #TOP
    top_noticias_seccion_el_pais = soup_pagina_seccion_el_pais.find('div', attrs={'class':'article-item__content'})
    # Obtener el link
    link_top_noticias_seccion_el_pais = [link_principal + top_noticias_seccion_el_pais.a.get('href')]
    #SEMI
    semi_noticias_seccion_el_pais = soup_pagina_seccion_el_pais.find_all('h3', attrs={'class':'title ff-26px-w800-ls-07'})
    link_semi_noticias_seccion_el_pais=[link_principal + link.a.get('href') for link in semi_noticias_seccion_el_pais]
    # ARTICULOS
    articulos_noticias_seccion_el_pais= soup_pagina_seccion_el_pais.find_all('h4', attrs={'class':'title is-display-inline ff-22px-w700-ls-07'})
    link_articulos_noticias_seccion_el_pais=[link_principal + link.a.get('href') for link in articulos_noticias_seccion_el_pais]

    URL.extend(link_top_noticias_seccion_el_pais)    
    URL.extend(link_semi_noticias_seccion_el_pais)
    URL.extend(link_articulos_noticias_seccion_el_pais)

    return URL

In [6]:
lista_notas = (obtener_notas(soup))
lista_notas

['https://www.pagina12.com.ar//597076-dolar-javier-milei-fue-denunciado-por-violencia-economica',
 'https://www.pagina12.com.ar//597118-patricia-bullrich-intento-sacar-provecho-de-la-trepada-del-d',
 'https://www.pagina12.com.ar//597119-la-centrales-sindicales-repudiaron-los-dichos-de-milei',
 'https://www.pagina12.com.ar//597001-milagro-sala-con-mejorias-notables-paso-de-cuidados-intensiv',
 'https://www.pagina12.com.ar//597121-cfk-y-los-riesgos-de-la-falta-de-regulacion-de-los-mercados-',
 'https://www.pagina12.com.ar//597008-la-utep-se-movilizo-en-apoyo-a-la-quita-del-iva',
 'https://www.pagina12.com.ar//597124-massa-el-que-especula-con-el-ahorro-de-la-gente-tiene-que-ir',
 'https://www.pagina12.com.ar//597126-la-alianza-de-luis-barrionuevo-con-javier-milei',
 'https://www.pagina12.com.ar//597059-la-corte-suprema-rechazo-las-recusaciones-presentadas-por-cf',
 'https://www.pagina12.com.ar//597127-cambiar-la-vida-en-nuestra-ciudad',
 'https://www.pagina12.com.ar//597069-una-asamblea-p

In [7]:
url_nota = lista_notas[0]
# Guarda la primera url
print(url_nota)

https://www.pagina12.com.ar//597076-dolar-javier-milei-fue-denunciado-por-violencia-economica


In [8]:
def obtener_info(s_nota):
    # Crea un diccionario vacío para poblarlo con la información
    ret_dict = {}
    # Extraer el titulo
    titulo = s_nota.find('h1')
    if titulo:
        ret_dict['titulo'] = titulo.text
    else:
        ret_dict['titulo'] = None
    # Extraer la fecha
    fecha = s_nota.find('time', attrs={'datetime': True})
    if fecha:
         ret_dict['fecha'] = fecha.get('datetime')
    else:
        ret_dict['fecha'] = None
    # Extraer copete
    copete = s_nota.find('h2', attrs={'class':'h4 ff-16px-w700'})
    if copete:
         ret_dict['copete'] = copete.get_text()
    else:
        ret_dict['copete'] = None
    # Exteraer volanta
    volanta = s_nota.find('h2', attrs={'class':'h3.ff-20px-w400'})
    if volanta:
         ret_dict['volanta'] = volanta.get_text()
    else:
        ret_dict['volanta'] = None
    # Extraer cuerpo
    cuerpo = s_nota.find('div', attrs={'class':'article-main-content article-text'})
    if cuerpo:
         ret_dict['cuerpo'] = cuerpo.get_text()
    else:
        ret_dict['cuerpo'] = None
    # Extraer autor
    autor = s_nota.find('div', attrs={'class':'author-name.ff-14px-w800'})
    if autor:
         ret_dict['autor'] = autor.get_text()
    else:
        ret_dict['autor'] = None

    # Extraer imagen
    media = s_nota.find_all('div', attrs={'class':'image-wrapper'})
    if len(media) == 0: # Si no se encontraron imágenes
        print('No se encontraron imágenes')
    else:
        imagen = media[0].find('img')
        if imagen:
            img_src = imagen.get('src')
            try:
                img_req = requests.get(img_src) # Realizar una solicitud solo para la imagen
                if img_req.status_code == 200:
                    ret_dict['imagen'] = img_req.content
                else:
                    ret_dict['imagen'] = None
            except Exception as e:
                print('No se puede obtener la imagen:', str(e))
        else:
            print('No se encontró una etiqueta de imagen en la clase "image-wrapper"')
    
    return ret_dict

    
    

In [9]:
def scrape_nota(url):
    try:
        nota = requests.get(url)
    except Exception as e:
        print('Error scrapendo URL', url)
        print(e)
        return None
    if nota.status_code !=200:
        print(f'Error obtenien nota {url}') # Imprime formateado el error
        print(f'status code = {nota.status_code}')
        return None

    s_nota = BeautifulSoup(nota.text, 'lxml')

    ret_dict = obtener_info(s_nota)
    ret_dict['url'] = url

    return ret_dict

In [10]:
url_nota

'https://www.pagina12.com.ar//597076-dolar-javier-milei-fue-denunciado-por-violencia-economica'

In [11]:
scrape_nota(url_nota) # llama la funcion Scrape_nota y le pasa el parametro url nota, devuelve un diccionario con toda la info del artículo

{'titulo': 'Dólar: Javier Milei fue denunciado por violencia económica',
 'fecha': '2023-10-11T00:01:00-03:00',
 'copete': 'El candidato a presidente pidió no renovar los plazos fijos',
 'volanta': None,
 'cuerpo': 'Los dichos del candidato presidencial de La Libertad Avanza Javier Milei que empujaron la corrida del dólar ilegal, por encima de los 1.000 pesos, tienen una primera demanda penal. La Cámara Federal de la Ciudad de Buenos Aires sorteará mañana la denuncia presentada por la abogada Valeria Carreras para que se abra una investigación por "incitación a la violencia económica o delito financiero" contra Milei.\xa0Carreras,\xa0representante de familiares de víctimas del submarino ARA San Juan, pidió en la presentación que sean citados a ampliar testimonio varios de los economistas que denunciaron las intenciones de Milei detrás de sus dichos, respecto del conocimiento de las consecuencias que tendrían y que fueron expresadas por él mismo en una entrevista con radio Mitre el lune

In [12]:
href_link_secciones


['https://www.pagina12.com.ar/secciones/el-pais',
 'https://www.pagina12.com.ar/secciones/economia',
 'https://www.pagina12.com.ar/secciones/sociedad',
 'https://www.pagina12.com.ar/suplementos/cultura-y-espectaculos',
 'https://www.pagina12.com.ar/secciones/deportes',
 'https://www.pagina12.com.ar/secciones/el-mundo',
 'https://www.pagina12.com.ar/secciones/ciencia']

In [13]:
# Crea un lista con las notas que estan en cada una de la secciones
notas = []
for link in href_link_secciones:
    try:
        r = requests.get(link)
        if r.status_code == 200:
            soup = BeautifulSoup(r.text, 'lxml')
            notas.extend(obtener_notas(soup))
        else:
            print('No se pudo obtener sección', link)
    except:
        print('No se pudo obener la sección', link)
        

In [14]:
notas


['https://www.pagina12.com.ar//597076-dolar-javier-milei-fue-denunciado-por-violencia-economica',
 'https://www.pagina12.com.ar//597118-patricia-bullrich-intento-sacar-provecho-de-la-trepada-del-d',
 'https://www.pagina12.com.ar//597119-la-centrales-sindicales-repudiaron-los-dichos-de-milei',
 'https://www.pagina12.com.ar//597001-milagro-sala-con-mejorias-notables-paso-de-cuidados-intensiv',
 'https://www.pagina12.com.ar//597121-cfk-y-los-riesgos-de-la-falta-de-regulacion-de-los-mercados-',
 'https://www.pagina12.com.ar//597008-la-utep-se-movilizo-en-apoyo-a-la-quita-del-iva',
 'https://www.pagina12.com.ar//597124-massa-el-que-especula-con-el-ahorro-de-la-gente-tiene-que-ir',
 'https://www.pagina12.com.ar//597126-la-alianza-de-luis-barrionuevo-con-javier-milei',
 'https://www.pagina12.com.ar//597059-la-corte-suprema-rechazo-las-recusaciones-presentadas-por-cf',
 'https://www.pagina12.com.ar//597127-cambiar-la-vida-en-nuestra-ciudad',
 'https://www.pagina12.com.ar//597069-una-asamblea-p

In [15]:
data = []
for i, nota in enumerate(notas):
    print(f'Scrapeando nota{i}/{len(notas)}')
    data.append(scrape_nota(nota))

Scrapeando nota0/84
Scrapeando nota1/84
Scrapeando nota2/84
Scrapeando nota3/84
Scrapeando nota4/84
Scrapeando nota5/84
Scrapeando nota6/84
Scrapeando nota7/84
Scrapeando nota8/84
Scrapeando nota9/84
Scrapeando nota10/84
Scrapeando nota11/84
Scrapeando nota12/84
Scrapeando nota13/84
Scrapeando nota14/84
Scrapeando nota15/84
Scrapeando nota16/84
Scrapeando nota17/84
Scrapeando nota18/84
Scrapeando nota19/84
Scrapeando nota20/84
Scrapeando nota21/84
Scrapeando nota22/84
Scrapeando nota23/84
Scrapeando nota24/84
Scrapeando nota25/84
Scrapeando nota26/84
Scrapeando nota27/84
Scrapeando nota28/84
Scrapeando nota29/84
Scrapeando nota30/84
Scrapeando nota31/84
Scrapeando nota32/84
Scrapeando nota33/84
Scrapeando nota34/84
Scrapeando nota35/84
Scrapeando nota36/84
Scrapeando nota37/84
Scrapeando nota38/84
Scrapeando nota39/84
Scrapeando nota40/84
Scrapeando nota41/84
Scrapeando nota42/84
Scrapeando nota43/84
Scrapeando nota44/84
Scrapeando nota45/84
Scrapeando nota46/84
Scrapeando nota47/84
Sc

In [16]:
len(data)

84

In [17]:
import pandas as pd

In [18]:
df = pd.DataFrame(data)

In [19]:
df.head

<bound method NDFrame.head of                                                titulo  \
0   Dólar: Javier Milei fue denunciado por violenc...   
1   Patricia Bullrich intentó sacar provecho de la...   
2   La centrales sindicales repudiaron los dichos ...   
3   Milagro Sala con "mejorías notables" pasó de c...   
4   CFK y los riesgos de la falta de regulación de...   
..                                                ...   
79    La alianza de Luis Barrionuevo con Javier Milei   
80  La Corte Suprema rechazó las recusaciones pres...   
81                  Cambiar la vida en nuestra ciudad   
82  Convocan a una "asamblea popular" frente a las...   
83  Marcelo D'Alessio se negó a declarar en el jui...   

                        fecha  \
0   2023-10-11T00:01:00-03:00   
1   2023-10-11T00:01:00-03:00   
2   2023-10-11T00:01:00-03:00   
3   2023-10-11T00:01:00-03:00   
4   2023-10-11T00:01:00-03:00   
..                        ...   
79  2023-10-11T00:01:00-03:00   
80  2023-10-11T00:01:

In [21]:
df.to_csv('Notas Pagina12.csv')